<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4ce88f60ed724f4de5cd3416efd3ffb6f0dcba9c98c2ab8f72bbf4ca8a8d56a1
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-13 12:41:45
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.50 L
Current:  1.45 C
-------------------
Today PnL: 4.45 K (0.03%)
Current PnL: -21.37 L (-14.0%)
CY Booked + Current PnL: -7.85 L (-5.14%)
-------------------
Total profit:  2.04 L
Total loss:  -23.41 L
-------------------
Total Booked + Current PnL: 19.13 L (15.23%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.31%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 88.04 L (60.6%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.45%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,102.13,52.0,M-SC,2.71,87095.0,-13682.0,13778.0,0.13,-13.58,15.82,0.09,245.0,-0.99,0.61,14.42,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.00,139.40,82.0,M-SC,3.24,156972.0,-17587.0,17628.0,2.71,-10.07,11.23,0.02,221.0,-1.00,1.09,36.54,OX40N,NTT,PAINTS
2,ABBOTINDIA,35195.00,-11.06,43.0,X-MC,3.39,87870.0,-2808.0,17715.0,-0.95,-3.10,20.16,16.44,101.0,-0.16,0.61,16.45,X40,ATH,PHARMA
34,ICICIGI,2252.93,-12.22,60.0,X-MC,7.59,189627.0,15307.0,19892.0,0.51,8.78,10.49,20.19,91.0,0.77,1.32,25.22,X40,ATH,INSURANCE
51,NESTLEIND,1377.00,-2.24,66.0,X-LC,8.76,300784.0,35358.0,21446.0,0.56,13.32,7.13,21.40,11.0,1.65,2.09,20.85,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.01,85.0,X-LC,7.96,252534.0,766.0,52451.0,3.61,0.30,20.77,21.13,27.0,0.01,1.76,35.84,X40,ATH,PAINTS
38,INDIGOPNTS,1408.00,139.40,82.0,M-SC,3.24,156972.0,-17587.0,17628.0,2.71,-10.07,11.23,0.02,221.0,-1.00,1.09,36.54,OX40N,NTT,PAINTS
5,ANGELONE,3033.00,27.04,74.0,X-SC,7.11,225090.0,34084.0,23612.0,2.44,17.84,10.49,30.21,157.0,1.44,1.57,42.45,X40N,NTT,FINANCE
46,KANSAINER,340.00,-67.24,56.0,H-SC,2.45,226350.0,-43317.0,79653.0,2.43,-16.06,35.19,13.47,138.0,-0.54,1.57,15.09,XY24,NTT,PAINTS
13,BERGEPAINT,680.00,-13.70,69.0,X-MC,1.58,231781.0,4442.0,43621.0,2.18,1.95,18.82,21.14,106.0,0.10,1.61,30.64,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.00,-54.55,54.0,H-SC,9.52,259993.0,-26512.0,283860.0,-4.29,-9.25,109.18,89.82,134.0,-0.09,1.81,37.56,XY24,NTT,FINANCE
15,CAMPUS,393.00,-26.26,40.0,M-SC,4.08,153630.0,-14145.0,69994.0,-2.03,-8.43,45.56,33.28,210.0,-0.20,1.07,19.07,XY24,NTT,FOOTWEAR
47,KPIGREEN,731.05,0.96,40.0,H-SC,10.23,120305.0,-4992.0,63918.0,-1.75,-3.98,53.13,47.03,141.0,-0.08,0.84,50.63,MH,ATH,POWER
7,ASIANTILES,137.00,7550.00,64.0,L-SC,15.83,83958.0,-9852.0,86468.0,-1.69,-10.50,102.99,81.67,269.0,-0.11,0.58,62.59,XR,NTT,CERAMICS
17,CERA,9475.00,-28.40,30.0,H-SC,2.27,130663.0,-45240.0,87257.0,-1.57,-25.72,66.78,23.89,149.0,-0.52,0.91,12.84,OX40N,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.34,55.0,M-LC,5.45,152235.0,1290.0,108163.0,0.13,0.85,71.05,72.51,53.0,0.01,1.06,6.93,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-9.48,38.0,H-MC,5.89,208933.0,-641.0,75320.0,-0.63,-0.31,36.05,35.63,NaN,-0.01,1.45,16.32,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIGOPNTS,1408.0,139.40,82.0,M-SC,3.24,156972.0,-17587.0,17628.0,2.71,-10.07,11.23,0.02,221.0,-1.00,1.09,36.54,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.0,102.13,52.0,M-SC,2.71,87095.0,-13682.0,13778.0,0.13,-13.58,15.82,0.09,245.0,-0.99,0.61,14.42,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.24,56.0,H-SC,2.45,226350.0,-43317.0,79653.0,2.43,-16.06,35.19,13.47,138.0,-0.54,1.57,15.09,XY24,NTT,PAINTS
66,SIS,528.0,2021.76,47.0,H-SC,2.14,85629.0,-25403.0,49014.0,-0.33,-22.88,57.24,21.26,156.0,-0.52,0.60,15.69,OX40N,NTT,MISC
72,TATAELXSI,9161.0,-24.89,47.0,H-MC,6.30,101660.0,-30175.0,72402.0,-0.70,-22.89,71.22,32.03,98.0,-0.42,0.71,13.87,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.34,55.0,M-LC,5.45,152235.0,1290.0,108163.0,0.13,0.85,71.05,72.51,53.0,0.01,1.06,6.93,XR,NTT,IT
37,INDIAMART,4810.62,-52.77,59.0,H-SC,4.77,132092.0,8756.0,122872.0,-0.49,7.10,93.02,106.72,119.0,0.07,0.92,32.14,AR,ATH,MISC
61,SAMMAANCAP,326.00,-193.18,56.0,M-SC,36.11,160695.0,11475.0,132702.0,0.89,7.69,82.58,96.62,208.0,0.09,1.12,73.91,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4810.62,-52.77,59.0,H-SC,4.77,132092.0,8756.0,122872.0,-0.49,7.10,93.02,106.72,119.0,0.07,0.92,32.14,AR,ATH,MISC
84,WIPRO,420.00,-13.34,55.0,M-LC,5.45,152235.0,1290.0,108163.0,0.13,0.85,71.05,72.51,53.0,0.01,1.06,6.93,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-9.48,38.0,H-MC,5.89,208933.0,-641.0,75320.0,-0.63,-0.31,36.05,35.63,NaN,-0.01,1.45,16.32,AR,ATH,PHARMA
83,WHIRLPOOL,2270.00,-45.97,35.0,M-SC,9.14,111992.0,-2806.0,96851.0,-0.78,-2.44,86.48,81.92,223.0,-0.03,0.78,32.58,XR,NTT,DURABLES
47,KPIGREEN,731.05,0.96,40.0,H-SC,10.23,120305.0,-4992.0,63918.0,-1.75,-3.98,53.13,47.03,141.0,-0.08,0.84,50.63,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GILLETTE,11206.78,-17.81,28.0,X-MC,5.78,252465.0,-1431.0,83743.0,-1.10,-0.56,33.17,32.42,105.0,-0.02,1.76,14.45,X40,ATH,FMCG
1,ABB,7934.00,-43.47,29.0,H-LC,5.17,236304.0,-25315.0,144524.0,-0.02,-9.68,61.16,45.57,7.0,-0.18,1.64,0.55,AR,NTT,ELECTRICAL
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87150.0,-41520.0,86819.0,-0.71,-32.27,99.62,35.20,219.0,-0.48,0.61,0.30,X40,NTT,FOOTWEAR
54,QUESS,986.00,-52.62,30.0,X-SC,37.27,47313.0,-17693.0,168624.0,-1.53,-27.22,356.40,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC
26,GLAXO,3466.20,-20.31,32.0,X-MC,5.30,201640.0,2356.0,75655.0,0.55,1.18,37.52,39.15,96.0,0.03,1.40,30.53,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,ITC,452.00,-38.34,46.0,X-LC,0.88,198925.0,-1213.0,22101.0,-0.05,-0.61,11.11,10.44,4.0,-0.05,1.38,4.92,X40,NTT,FMCG
53,PGHH,17907.65,-32.93,33.0,X-MC,1.54,194250.0,-6570.0,74359.0,0.61,-3.27,38.28,33.76,80.0,-0.09,1.35,1.76,X40,ATH,FMCG
65,SIEMENS,4671.50,-3.90,47.0,H-LC,1.64,154360.0,-31735.0,79218.0,1.00,-17.05,51.32,25.51,15.0,-0.40,1.07,13.75,AR,ATH,ELECTRICAL
30,HAVELLS,2069.16,-7.60,45.0,X-MC,3.21,308154.0,-19837.0,126374.0,0.64,-6.05,41.01,32.48,92.0,-0.16,2.14,4.69,X40,ATH,ELECTRICAL
78,UNITDSPR,1644.00,-6.09,62.0,X-LC,3.28,246132.0,11988.0,36624.0,-0.32,5.12,14.88,20.77,86.0,0.33,1.71,12.39,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-52.62,30.0,X-SC,37.27,47313.0,-17693.0,168624.0,-1.53,-27.22,356.40,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87150.0,-41520.0,86819.0,-0.71,-32.27,99.62,35.20,219.0,-0.48,0.61,0.30,X40,NTT,FOOTWEAR
1,ABB,7934.00,-43.47,29.0,H-LC,5.17,236304.0,-25315.0,144524.0,-0.02,-9.68,61.16,45.57,7.0,-0.18,1.64,0.55,AR,NTT,ELECTRICAL
53,PGHH,17907.65,-32.93,33.0,X-MC,1.54,194250.0,-6570.0,74359.0,0.61,-3.27,38.28,33.76,80.0,-0.09,1.35,1.76,X40,ATH,FMCG
10,BAJAJHFL,181.50,-16.17,38.0,X-MC,6.42,176841.0,-23665.0,120818.0,-0.19,-11.80,68.32,48.45,90.0,-0.20,1.23,1.98,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-52.62,30.0,X-SC,37.27,47313.0,-17693.0,168624.0,-1.53,-27.22,356.40,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-40.75,42.0,X-SC,3.58,80765.0,-64395.0,140321.0,-1.15,-44.36,173.74,52.31,136.0,-0.46,0.56,8.11,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-11.06,43.0,X-MC,3.39,87870.0,-2808.0,17715.0,-0.95,-3.10,20.16,16.44,101.0,-0.16,0.61,16.45,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87150.0,-41520.0,86819.0,-0.71,-32.27,99.62,35.20,219.0,-0.48,0.61,0.30,X40,NTT,FOOTWEAR
33,HONAUT,58357.33,-24.48,35.0,X-SC,0.97,106395.0,-21543.0,68678.0,-0.08,-16.84,64.55,36.84,143.0,-0.31,0.74,8.92,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-26.44,62.0,X-LC,8.77,293252.0,-52704.0,119412.0,-0.39,-15.23,40.72,19.28,1.0,-0.44,2.04,8.41,X40,ATH,IT
40,INFY,2275.00,-15.29,66.0,X-LC,4.32,329894.0,17006.0,154687.0,-0.19,5.44,46.89,54.87,3.0,0.11,2.29,14.17,X40,BTT,IT
42,ITC,452.00,-38.34,46.0,X-LC,0.88,198925.0,-1213.0,22101.0,-0.05,-0.61,11.11,10.44,4.0,-0.05,1.38,4.92,X40,NTT,FMCG
81,VBL,671.64,-18.84,46.0,X-LC,5.85,292364.0,-23478.0,136154.0,-0.23,-7.43,46.57,35.67,5.0,-0.17,2.03,5.39,X40N,ATH,FMCG
1,ABB,7934.00,-43.47,29.0,H-LC,5.17,236304.0,-25315.0,144524.0,-0.02,-9.68,61.16,45.57,7.0,-0.18,1.64,0.55,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1800.00,307.56,80.0,L-MC,6.68,50006.0,-27200.0,52591.0,1.49,-35.23,105.17,32.89,258.0,-0.52,0.35,37.71,XR,NTT,BANKS
19,COFFEEDAY,80.00,-47.69,47.0,L-SC,22.27,78336.0,-35213.0,75265.0,0.87,-31.01,96.08,35.27,268.0,-0.47,0.54,91.73,XR,NTT,HOTELS
7,ASIANTILES,137.00,7550.00,64.0,L-SC,15.83,83958.0,-9852.0,86468.0,-1.69,-10.50,102.99,81.67,269.0,-0.11,0.58,62.59,XR,NTT,CERAMICS
50,MASFIN,398.61,-18.70,50.0,H-SC,7.52,92145.0,-5835.0,27441.0,-0.28,-5.96,29.78,22.05,152.0,-0.21,0.64,33.94,XR,ATH,FINANCE
37,INDIAMART,4810.62,-52.77,59.0,H-SC,4.77,132092.0,8756.0,122872.0,-0.49,7.10,93.02,106.72,119.0,0.07,0.92,32.14,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.01,85.0,X-LC,7.96,252534.0,766.0,52451.0,3.61,0.30,20.77,21.13,27.0,0.01,1.76,35.84,X40,ATH,PAINTS
79,VAIBHAVGBL,521.00,73.55,62.0,H-SC,7.58,150616.0,-32159.0,143221.0,0.81,-17.60,95.09,60.77,125.0,-0.22,1.05,35.64,XR,NTT,JEWELLERY
38,INDIGOPNTS,1408.00,139.40,82.0,M-SC,3.24,156972.0,-17587.0,17628.0,2.71,-10.07,11.23,0.02,221.0,-1.00,1.09,36.54,OX40N,NTT,PAINTS
7,ASIANTILES,137.00,7550.00,64.0,L-SC,15.83,83958.0,-9852.0,86468.0,-1.69,-10.50,102.99,81.67,269.0,-0.11,0.58,62.59,XR,NTT,CERAMICS
5,ANGELONE,3033.00,27.04,74.0,X-SC,7.11,225090.0,34084.0,23612.0,2.44,17.84,10.49,30.21,157.0,1.44,1.57,42.45,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.84
1,25,44.64
2,50,76.31


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.45
LC    28.00
MC    27.08
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.42
X40      20.85
X40N     12.41
XY25     10.16
XR        9.74
AR        9.10
OX40N     6.58
X200      1.84
SR        1.04
MH        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.85
X-MC    22.96
X-LC    20.02
M-SC    12.52
X-SC     5.63
H-LC     4.55
H-MC     3.77
M-LC     2.44
L-SC     1.45
L-LC     0.99
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.81,-4.10,37.77
IT,12.77,-14.44,75.69
FINANCE,12.06,-9.04,58.47
MISC,7.69,-17.73,77.11
PAINTS,6.03,-6.42,22.29
ELECTRICAL,5.59,-12.22,52.01
INSURANCE,4.51,2.34,32.08
PHARMA,3.46,-0.22,33.84
AUTO,3.17,-52.85,116.14


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3222153.0,20
AR,1262791.0,10
XR,1255428.0,13
X40,972349.0,14
X40N,684974.0,9
OX40N,539629.0,9
XY25,472909.0,7
SR,270544.0,2
MH,63918.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3401963.0,24
X-MC,1386475.0,16
M-SC,1245988.0,15
X-LC,772677.0,11
X-SC,541221.0,6
H-MC,329866.0,3
H-LC,282597.0,3
L-SC,251025.0,3
M-LC,154695.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1177065.0      6
           AR         839759.0      5
           XR         788321.0      7
M-SC       XY24       729117.0      6
X-MC       XY24       553854.0      4
           X40        486253.0      8
X-LC       X40        399277.0      5
X-SC       X40N       285778.0      4
H-SC       SR         270544.0      2
           OX40N      262356.0      3
X-LC       X40N       238344.0      3
H-LC       AR         223742.0      2
X-MC       XY25       185516.0      2
H-MC       XY24       182144.0      1
X-SC       XY24       168624.0      1
L-SC       XR         161733.0      2
X-MC       X40N       160852.0      2
M-SC       XR         144620.0      2
           XY25       132702.0      1
           AR         123970.0      2
           OX40N      115579.0      4
M-LC       XR         108163.0      1
L-SC       OX40N       89292.0      1
X-SC       X40         86819.0      1
H-MC       AR          75320.0      1
           OX40N       72402.0      1
X-LC       XY25        71597.0      2
H-SC       MH          63918.0      1
X-LC       XY24        63459.0      1
H-LC       X200        58855.0      1
L-MC       XR          52591.0      1
M-LC       XY25        46532.0      1
L-LC       XY25        36562.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
